# Telco Customer Churn Analysis - Data Preparation

## Project Overview

This notebook performs data cleaning and exploratory data analysis for the Telco Customer Churn dataset. We'll prepare the data for machine learning modeling to predict customer churn and identify high-risk customers.

### Goals:
- Clean and standardize the raw dataset
- Handle missing values appropriately
- Understand data distribution and quality
- Prepare clean dataset for modeling

### Dataset:
- **Source**: Kaggle Telco Customer Churn Dataset
- **Target Variable**: Churn (Yes/No)
- **Features**: Customer demographics, services, and billing information

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Setup paths
ROOT = Path("..").resolve()
DATA = ROOT / "data"
OUTF = ROOT / "reports" / "figures"
OUTF.mkdir(parents=True, exist_ok=True)
OUTT = ROOT / "reports" / "tables"
OUTT.mkdir(parents=True, exist_ok=True)

print(f"Project root: {ROOT}")
print(f"Data directory: {DATA}")
print(f"Output directories created successfully")

[WinError 3] The system cannot find the path specified: 'C:\\Users\\seewi/Projects/Churn-Telco/notebooks'
c:\Workspaces\VScode\Portfolio_Projects\telco-customer-churn\notebooks
(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Columns with missing values:
               num_missing  pct_missing
totalcharges           11     0.001562
churn
No     0.73463
Yes    0.26537
Name: share, dtype: float64
✅ saved: C:\Workspaces\VScode\Portfolio_Projects\telco-customer-churn\data\telco__customer_churn_clean.csv


In [ ]:
# Load the dataset
# Dataset source: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
df = pd.read_csv(DATA / "WA_Fn-UseC_-Telco-Customer-Churn.csv")

print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nFirst few rows:")
display(df.head())

## 2. Data Cleaning and Standardization

In [ ]:
# Standardize column names (lowercase, replace spaces with underscores)
print("Original column names:")
print(df.columns.tolist())

df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

print("\nStandardized column names:")
print(df.columns.tolist())

In [ ]:
# Define target variable and handle data type issues
target = "churn"  # Target variable: "Yes"/"No"

# Ensure target is properly formatted
if df[target].dtype != "O":
    df[target] = df[target].map({1: "Yes", 0: "No"})

print(f"Target variable: {target}")
print(f"Target distribution:")
print(df[target].value_counts())

# Handle TotalCharges column (often has string formatting issues)
if "totalcharges" in df.columns:
    print(f"\nOriginal TotalCharges data type: {df['totalcharges'].dtype}")
    print(f"Sample TotalCharges values: {df['totalcharges'].head().tolist()}")
    
    # Convert TotalCharges to numeric (replace spaces with NaN)
    df["totalcharges"] = pd.to_numeric(df["totalcharges"].replace(" ", np.nan), errors="coerce")
    
    print(f"After conversion - TotalCharges data type: {df['totalcharges'].dtype}")
    print(f"TotalCharges missing values: {df['totalcharges'].isna().sum()}")

## 3. Missing Value Analysis and Treatment

In [ ]:
# Comprehensive missing value analysis
nulls = df.isna().mean().sort_values(ascending=False)
miss_cols = nulls[nulls > 0].index.tolist()

print("Missing Value Analysis:")
print("=" * 50)

if miss_cols:      
    miss_df = pd.DataFrame({
        "column": miss_cols,
        "num_missing": df[miss_cols].isna().sum(),
        "pct_missing": df[miss_cols].isna().mean() * 100
    }).sort_values("pct_missing", ascending=False)
    
    print("Columns with missing values:")
    display(miss_df)
    
    # Visualize missing values
    import matplotlib.pyplot as plt
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Missing values count
    miss_df.plot(x="column", y="num_missing", kind="bar", ax=ax1, color="coral")
    ax1.set_title("Missing Values Count by Column")
    ax1.set_xlabel("Columns")
    ax1.set_ylabel("Number of Missing Values")
    ax1.tick_params(axis='x', rotation=45)
    
    # Missing values percentage
    miss_df.plot(x="column", y="pct_missing", kind="bar", ax=ax2, color="skyblue")
    ax2.set_title("Missing Values Percentage by Column")
    ax2.set_xlabel("Columns")
    ax2.set_ylabel("Percentage Missing (%)")
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(OUTF / "missing_values_analysis.png", dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("✅ No missing values found in the dataset!")

In [ ]:
# Handle missing values in numerical columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print("Numerical columns for imputation:")
print(num_cols)

# Simple imputation: fill missing numerical values with median
missing_before = df[num_cols].isna().sum().sum()

for col in num_cols:
    if df[col].isna().any():
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"Filled {col} missing values with median: {median_val}")

missing_after = df[num_cols].isna().sum().sum()

print(f"\nMissing values in numerical columns:")
print(f"Before imputation: {missing_before}")
print(f"After imputation: {missing_after}")
print(f"✅ Successfully imputed {missing_before - missing_after} missing values")

## 4. Exploratory Data Analysis

In [ ]:
# Analyze target variable distribution (class balance)
print("TARGET VARIABLE ANALYSIS")
print("=" * 50)

churn_counts = df[target].value_counts()
churn_proportions = df[target].value_counts(normalize=True)

print("Absolute counts:")
display(churn_counts)
print("\nProportions:")
display(churn_proportions.round(4))

# Calculate churn rate
churn_rate = churn_proportions.get('Yes', 0) * 100
print(f"\n📊 Overall Churn Rate: {churn_rate:.1f}%")

# Visualize class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Count plot
churn_counts.plot(kind='bar', ax=ax1, color=['lightgreen', 'coral'])
ax1.set_title('Churn Distribution (Counts)')
ax1.set_xlabel('Churn Status')
ax1.set_ylabel('Number of Customers')
ax1.tick_params(axis='x', rotation=0)

# Proportion pie chart
ax2.pie(churn_proportions.values, labels=churn_proportions.index, autopct='%1.1f%%', 
        colors=['lightgreen', 'coral'], startangle=90)
ax2.set_title('Churn Distribution (Proportions)')

plt.tight_layout()
plt.savefig(OUTF / "churn_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

# Class imbalance assessment
if churn_rate < 40:
    print("⚠️  Class imbalance detected - will need to address in modeling")
else:
    print("✅ Classes are reasonably balanced")

In [ ]:
# Dataset overview and data types
print("DATASET OVERVIEW")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nData Types Summary:")
dtype_summary = df.dtypes.value_counts()
display(dtype_summary)

print("\nColumn Information:")
info_df = pd.DataFrame({
    'Column': df.columns,
    'Data Type': df.dtypes.values,
    'Non-Null Count': df.count().values,
    'Null Count': df.isnull().sum().values,
    'Unique Values': [df[col].nunique() for col in df.columns]
})
display(info_df)

## 5. Save Cleaned Dataset

In [ ]:
# Save the cleaned dataset
clean_path = DATA / "telco__customer_churn_clean.csv"
df.to_csv(clean_path, index=False)

print("DATA PREPARATION COMPLETED")
print("=" * 50)
print(f"✅ Cleaned dataset saved to: {clean_path}")
print(f"📊 Final dataset shape: {df.shape}")
print(f"🎯 Target variable: {target}")
print(f"📈 Churn rate: {churn_rate:.1f}%")
print(f"🔧 Missing values handled: {missing_before} → {missing_after}")

# Final data quality check
print(f"\nFinal Data Quality Summary:")
print(f"- Total records: {len(df):,}")
print(f"- Total features: {len(df.columns)}")
print(f"- Numerical features: {len(num_cols)}")
print(f"- Categorical features: {len(df.columns) - len(num_cols)}")
print(f"- Missing values remaining: {df.isnull().sum().sum()}")
print(f"- Duplicate rows: {df.duplicated().sum()}")

print(f"\n🚀 Ready for modeling in 02_modeling.ipynb")

## Summary and Conclusions

### Data Preparation Results

✅ **Data Loading**: Successfully loaded the Telco Customer Churn dataset  
✅ **Data Cleaning**: Standardized column names and handled data type issues  
✅ **Missing Values**: Identified and imputed missing values using median imputation  
✅ **Quality Check**: Validated data integrity and format consistency  

### Key Findings

1. **Dataset Characteristics**:
   - Total customers analyzed
   - Features available for modeling
   - Data quality is high with minimal missing values

2. **Class Distribution**:
   - Churn rate identified (class imbalance consideration for modeling)
   - Baseline understanding of customer retention

3. **Data Quality**:
   - All missing values handled appropriately
   - Data types standardized for modeling
   - No duplicate records found

### Next Steps

The cleaned dataset is ready for:
1. **Baseline Modeling** (02_modeling.ipynb)
2. **Feature Engineering** (03_feature_engineering_and_tuning.ipynb)
3. **Advanced Analytics and Visualization**

### Files Generated

- **Clean Dataset**: `telco__customer_churn_clean.csv`
- **Visualizations**: Missing value analysis, churn distribution plots

---
**Status**: ✅ Data preparation completed successfully